# Multi-model Systems with the OpenAI API

In [1]:
import sys
import os

# Use current working directory and go one level up
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

# Now you can import your config
from config import api_key

## Chapter 1 - Beyond-to-text

### Section 1.1 - Speech-to-text

#### Creating a podcast transcript

The OpenAI API Audio endpoint provides access to the different models, which can be used for speech-to-text transcription and translation. In this exercise, you'll create a transcript from a DataFramed podcast episode with OpenAI Developer, Logan Kilpatrick.

If you'd like to hear more from Logan, check out the full ChatGPT and the OpenAI Developer Ecosystem podcast episode.

In [4]:
from openai import OpenAI

client = OpenAI(api_key=api_key)

# Open the openai-audio.mp3 file
audio_file = open("audio-logan-advocate-openai.mp3", "rb")

# Create a transcript from the audio file
response = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

# Extract and print the transcript text
print(response.text)

Hi there, Logan, thank you for joining us on the show today. Thanks for having me. I'm super excited about this. Brilliant. We're going to dive right in, and I think ChatGPT is maybe the most famous AI product that you have at OpenAI, but I'd just like to get an overview of what all the other AIs that are available are. So I think two and a half years ago, OpenAI released the API that we still have available today, which is essentially our giving people access to these models. And for a lot of people, giving people access to the model that powers ChatGPT, which is our consumer-facing first-party application, which essentially just, in very simple terms, puts a nice UI on top of what was already available through our API for the last two and a half years. So it's sort of democratizing the access to this technology through our API. And if you want to just play around with it as an end user, we have ChatGPT available to the world as well.


#### Transcribing a non-English language

OpenAI's audio models can not only transcribe English speech but also perform well in speech in many other languages.

In this exercise, you'll create a transcript from audio.m4a, which contains speech in Portuguese.

In [9]:
client = OpenAI(api_key=api_key)

# Open the audio.m4a file
audio_file = open("audio-portuguese.m4a", "rb")

# Create a transcript from the audio file
response = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

print(response.text)

Olá, o meu nome é Eduardo, sou CTO no Datacamp. Espero que esteja a gostar deste curso que o James e eu criamos para você. Esta API permite enviar um áudio e trazer para inglês. O áudio original está em português.


#### Translating Portuguese

OpenAI's audio models can not only transcribe audio into its native language, but also support translation capabilities for creating English transcriptions.

In this exercise, you'll return to the Portuguese audio, but this time, you'll translate it into English!

In [11]:
client = OpenAI(api_key=api_key)

# Open the audio.m4a file
audio_file = open('audio-portuguese.m4a', "rb")

# Create a translation from the audio file
response = client.audio.translations.create(model="whisper-1", file=audio_file)

# Extract and print the translated text
print(response.text)

Hello, my name is Eduardo, I am a CTO at Datacamp. I hope you are enjoying this course that James and I have created for you. This API allows you to send an audio and bring it to English. The original audio is in Portuguese.


### Section 1.2 - Text-to-speech (TTS)

#### OpenAI's text-to-speech (TTS)
OpenAI now provide models for creating human-like speech from a text input, so-called text-to-speech or TTS. OpenAI provide several voices to choose from, and they provide the ability to stream the response to local files or downstream applications.

In [21]:
client = OpenAI(api_key=api_key)

# Create the text-to-speech request
response = client.audio.speech.create(
  model="gpt-4o-mini-tts",
  voice="ballad",
  input="Hi! How's your day going?"
)

# Stream the response to an MP3 file
response.stream_to_file("output.mp3")

/tmp/ipykernel_286523/3595893031.py:11: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("output.mp3")


In [18]:
response.write_to_file('test.mp3')

#### TTS in other languages!
Let's have a go at some non-English text. Try sending the following text input to the model:

`Dnes je krásný slunečný den.`

The text is in the Czech language, which is spoken primarily in the Czech Republic.

In [22]:
client = OpenAI(api_key=api_key)

# Pass the non-English text to the model
response = client.audio.speech.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="Dnes je krásný slunečný den."
)

response.stream_to_file("output.mp3")

/tmp/ipykernel_286523/3058821576.py:10: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("output.mp3")


### Section 1.3 - Content moderation

#### Requesting moderation
Aside from text and chat completion models, OpenAI provides models with other capabilities, including text moderation. OpenAI's text moderation model is designed for evaluating prompts and responses to determine if they violate OpenAI's usage policies, including inciting hate speech and promoting violence.

In this exercise, you'll test out OpenAI's moderation functionality on a sentence that may have been flagged as containing violent content using traditional word detection algorithms.

In [24]:
client = OpenAI(api_key=api_key)

# Create a request to the Moderation endpoint
response = client.moderations.create(
    model="text-moderation-latest",
    input="My favorite book is To Kill a Mockingbird."
)

# Print the category scores
print(response.results[0].category_scores)

CategoryScores(harassment=5.243551186140394e-06, harassment_threatening=1.1516095810293336e-06, hate=4.767837526742369e-05, hate_threatening=3.2021056028952444e-08, illicit=None, illicit_violent=None, self_harm=9.466615438213921e-07, self_harm_instructions=5.426785065765216e-08, self_harm_intent=1.5536235764557205e-07, sexual=3.545879735611379e-06, sexual_minors=1.1304399549771915e-06, violence=0.0001064608441083692, violence_graphic=1.086988686438417e-05, self-harm=9.466615438213921e-07, sexual/minors=1.1304399549771915e-06, hate/threatening=3.2021056028952444e-08, violence/graphic=1.086988686438417e-05, self-harm/intent=1.5536235764557205e-07, self-harm/instructions=5.426785065765216e-08, harassment/threatening=1.1516095810293336e-06)


In [29]:
response.results[0].flagged # none of the categories flags True

False

#### Examining moderation category scores

The same request you created in the last exercise to the Moderation endpoint has been run again, sending the sentence "My favorite book is To Kill a Mockingbird." to the model. The response from the API has been printed for you, and is available as response.

Extract the category scores to determine the correct interpretation from the following list of statements.

## Chapter 2 - Creating customer call transcripts

### Section 2.1 - Creating customer call transcripts

#### Transcribing customer calls
You're working as an AI Engineer at DataCamp, which has an international learner base spanning 180+ countries. Their customer support team want to test receiving customer queries via phone call; however, they need your support to design a system to help their agents handle these requests.

To start, you will use the OpenAI API to transcribe their customer calls in the language they were recorded in.

As this is a case study, less code will be provided for you than in the previous chapter. If you struggle, don't give up, you can do this!

In [32]:
client = OpenAI(api_key=api_key)

# Open the customer_call.wav file
audio_file = open("customer_call_it_v2.wav","rb")

# Create a transcript from the audio file
response = client.audio.transcriptions.create(
    model='whisper-1',
    file=audio_file
)
transcript = response.text
print(transcript)

Ciao! Vorrei imparare l'AI con DataCamp, ma cosa posso usare? PyTorch, l'API di OpenAI, Lungchain o qualcos'altro?


#### Deriving the customer's language

Now that you have a transcript of the customer call, you need to derive the language used to eventually translate the response back to the customer in their native language. You'll use an OpenAI chat model to identify the language.

The output from the model should be standardized to a country code; for example, "en" for English, "fr" for French, and so on.

The transcript you created in the previous exercise is available as transcript.

In [34]:
client = OpenAI(api_key=api_key)

# Complete a request identify the country code from the transcript
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", 
    "content":f"Indentify the language of the following text and response only with the country code (e.g 'en', 'nl', 'uk', 'fr': {transcript}"}]
)

# Extract the country code from the response
country_code = response.choices[0].message.content
print(country_code)

it


#### Translating the transcript into English

With the customer call transcript created, it's time to translate it into English for processing.

The `transcript` you created previously is available as transcript, and the country code is stored in `country_code`.

In [36]:
client = OpenAI(api_key=api_key)

# Translate the transcript into English
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user",
        "content":f"Please translate the following transcript: {transcript} from the country code {country_code} into English"}]
 
)

# Extract the translated transcript text
en_transcript = response.choices[0].message.content
print(en_transcript)

Hello! I would like to learn AI with DataCamp, but what can I use? PyTorch, the OpenAI API, Langchain, or something else?


#### Fixing translation mistakes
As you saw, translations aren't always perfect, particularly if the transcript contains names or references outside of the model's knowledge. In this exercise, you'll use another call to the chat model to fix these incorrect words.

The English transcript you created in the previous exercise is available as `en_transcript`.

In [38]:
client = OpenAI(api_key=api_key)

# Fix the mistake in the transcript
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": f"You are an AI assistant that corrects transcripts by fixing names and terminology, leaving the rest of the transcript unchanged. Please refine the following transcript: {en_transcript}"}]
)

corrected_text = response.choices[0].message.content
print(corrected_text)

Hello! I would like to learn AI with DataCamp, but what can I use? PyTorch, the OpenAI API, LangChain, or something else?


### Section 2.2 Generating a customer response

#### Moderating the customer call
To ensure quality and safety, you'll now moderate the processed customer message. This step helps identify and flag potentially inappropriate content such as harassment.

The corrected transcript from the previous exercise is available as `corrected_text`.

In [47]:
client = OpenAI(api_key=api_key)

# Send the moderation request
response = client.moderations.create(
    model='text-moderation-latest',
    input=corrected_text
)

# Extract scores and convert to dictionary
scores = response.results[0].category_scores.model_dump()

# Extract harassment score
harassment_score = scores['harassment']

if harassment_score>0.8:
    print("Content flagged for harassment!")
else:
    print("Content is safe from harassment.")

Content is safe from harassment.


#### Generating a tailored response
Now that the customer inquiry passed moderation, it's time to generate a tailored response.

To help the chatbot respond accurately, you'll provide it with relevant context. Two documents are available:

- faqs: common customer questions and answers
- content_overview: relevant tracks, including descriptions and links

The corrected transcript is still available as `corrected_text`

In [50]:
faqs = "\nQ: How can I upgrade my subscription?  \nA: Log into your account, go to 'Account Settings' > 'Subscription,' and click 'Upgrade.' You will only pay the difference if upgrading from monthly to yearly. //\n\nQ: How do I contact DataCamp Customer Support?  \nA: Use the live chat button in the bottom-right corner of the Help Center for 24/7 assistance. Alternatively, email support@datacamp.com or use the contact form. //\n\nQ: Does DataCamp offer content in multiple languages?  \nA: Yes, many courses are available in languages like Spanish, Portuguese, German, and French. You can change the language in your account settings. //\n\nQ: Why did I lose access to my DataCamp Classroom?  \nA: Classrooms are valid for six months. Reapply for a new Classroom to regain access. //\n\nQ: Can I integrate Data Connector with BI tools like PowerBI or Tableau?  \nA: Yes, it integrates with tools like PowerBI, Tableau, and Looker. //\n\nQ: Can I invite non-teaching staff to my DataCamp Classroom?  \nA: No, only teaching staff and students may join Classrooms. Non-teaching staff should apply for their own accounts. //\n\nQ: How do I prepare for a course before my Classroom starts?  \nA: Use a free account to access the first chapter of courses or set an earlier start date during application. //\n\nQ: What happens if I cancel my subscription before upgrading?  \nA: You will be charged the full yearly rate unless you reactivate your monthly subscription first by contacting Support. //\n\nQ: Where can I download all the course content?  \nA: DataCamp does not allow downloading course content, but you can access it anytime online. //\n\nQ: Does DataCamp provide certification for Microsoft's AZ-900?  \nA: Yes, DataCamp offers preparation for Microsoft's AZ-900 Certification through specialized courses. //\n\nQ: Can I earn certification for Microsoft’s PL-300 on DataCamp?  \nA: Yes, DataCamp provides courses to prepare for Microsoft’s PL-300 Certification. //\n\nQ: What should I do if I experience video playback issues during a course?  \nA: Check your internet connection, disable browser extensions, and clear your cache. If the issue persists, contact Support. //\n\nQ: Will I receive a grade or bonus XP when I finish a course?  \nA: No grades are given, but you will earn XP upon completing a course. Bonus XP may apply during special events. //\n\nQ: Why is my XP decreasing?  \nA: XP may decrease if content is updated or removed from the platform. Your overall progress remains unaffected. //\n\nQ: Can I reset my account's XP and progress?  \nA: No, resetting XP and progress is not possible on DataCamp. //\n\nQ: Can I use XP to redeem content on DataCamp?  \nA: No, XP is only used to track learning progress and cannot be redeemed for content. //\n\nQ: How can I share my finished DataCamp projects?  \nA: You can download your completed projects as PDFs or share them via a public link provided by DataCamp. //\n\nQ: How much does a DataCamp statement of accomplishment cost?  \nA: Statements of accomplishment are free and automatically available after completing a course or track. //\n\nQ: How can I share my statement of accomplishment on LinkedIn?  \nA: Download your statement as a PDF and upload it to LinkedIn under 'Licenses & Certifications.' Include the issuing organization as 'DataCamp.' //\n\nQ: Can I update my name on a statement of accomplishment?  \nA: Yes, update your name in your account settings before downloading the statement again. //\n\nQ: What are recent updates to DataCamp’s content?  \nA: Recent updates include new courses and features released quarterly, such as Q1 2025 updates covering advanced Python topics and machine learning tools. //\n\nQ: What is DataCamp Certification?  \nA: DataCamp Certification is an official recognition of your skills as a Data Scientist, Data Analyst, or Data Engineer. It includes timed exams and a practical case study based on real-world scenarios. //\n\nQ: Do I need a subscription to access certification?  \nA: Yes, certification is available to individual subscribers and some business accounts. //\n\nQ: What languages are supported for certifications?  \nA: Certifications are currently only available in English. //\n\nQ: Do I need to know both R and Python for certification?  \nA: No, you can choose to complete your exams in either R or Python. //\n...\n"
content_overview = "\nContent Type: Career Track // Title: Associate AI Engineer for Developers // Description: Begin integrating AI into software applications and gain the career-building skills you need to succeed as an AI Engineer! You'll use Large Language Models (LLMs), prompt engineering, chatbots, recommendation engines, and vector databases. Explore OpenAI API, Hugging Face, LangChain, and Pinecone. // Technology: Python // Duration: 30 hours // Released: Aug 14, 2024 // URL: https://www.datacamp.com/tracks/associate-ai-engineer-for-developers\n\nContent Type: Skills Track // Title: Llama Fundamentals // Description: This track equips you with the skills to implement Llama 3 for tasks like summarization, text generation, and model fine-tuning. Explore how to integrate open-source Llama models into various projects. // Technology: Llama // Duration: 5 hours // Released: Dec 20, 2024 // URL: https://www.datacamp.com/tracks/llama-fundamentals\n\nContent Type: Skills Track // Title: OpenAI Fundamentals // Description: Learn to use the OpenAI API in Python to create AI applications like chatbots, semantic search, and recommendation systems. Hands-on coding exercises include advanced prompting techniques and embedding models. // Technology: OpenAI // Duration: 13 hours // Released: Jun 13, 2024 // URL: https://www.datacamp.com/tracks/openai-fundamentals\n\nContent Type: Skills Track // Title: Deep Learning in Python // Description: Dive into deep learning with PyTorch! Build models for real-world tasks, including electricity consumption prediction, cloud classification, and language identification. Work with CNNs, RNNs, and Transformers. // Technology: PyTorch // Duration: 18 hours // Released: Mar 13, 2025 // URL: https://www.datacamp.com/tracks/deep-learning-in-python\n\nContent Type: Skills Track // Title: ChatGPT Fundamentals // Description: Master the basics of ChatGPT, effective prompting, and workflow automation. Learn how businesses use ChatGPT to optimize operations and enhance AI-driven solutions. // Technology: ChatGPT // Duration: 3 hours // Released: Sep 20, 2024 // URL: https://www.datacamp.com/tracks/chatgpt-fundamentals\n...\n"


In [51]:
client = OpenAI(api_key=api_key)

# Complete the prompt
instruction_prompt = f"""
#### Role
You are a professional AI support assistant for DataCamp. You help with:
- Sales (pricing, plans, billing)
- Content (courses, recommendations, feedback)
- Marketing (partnerships, collaborations)

#### How to Respond
1. Use the FAQs: {faqs}
2. Use the content overview: {content_overview} 
3. Respond clearly and concisely in up to 3 paragraphs.
4. If unsure, direct the user to support@datacamp.com.
"""

# Generate response
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role":"system","content": instruction_prompt},
    {"role":"user", "content":corrected_text}]
)

chatbot_reply = response.choices[0].message.content
print(chatbot_reply)

DataCamp offers a variety of courses and tracks to help you learn AI, depending on your interests and goals. Here are a few options you can consider:

1. **Associate AI Engineer for Developers**: This career track focuses on integrating AI into software applications, covering Large Language Models (LLMs), prompt engineering, chatbots, and more. You'll also get to explore tools like OpenAI API, Hugging Face, LangChain, and Pinecone. It has a duration of 30 hours and is designed to equip you with the necessary skills to succeed as an AI Engineer. You can find this track [here](https://www.datacamp.com/tracks/associate-ai-engineer-for-developers).

2. **OpenAI Fundamentals**: If you're specifically interested in using the OpenAI API, this skills track teaches you how to create AI applications such as chatbots and recommendation systems, incorporating advanced prompting techniques as well. This track lasts for 13 hours, and you can explore it [here](https://www.datacamp.com/tracks/openai-f

In [54]:
client = OpenAI(api_key=api_key)

# Send the moderation request
response = client.moderations.create(
    model="text-moderation-latest",
    input=chatbot_reply
)

# Extract scores and convert to dictionary
scores = response.results[0].category_scores.model_dump()
print(scores)

if all(score>0.7 for score in scores.values()):
    print("AI Response flagged for moderation!")
    chatbot_reply = """I'm sorry, but I can't provide a response to that request. Please contact support@datacamp.com for further assistance."""
else:
    print("AI Response is safe.")

{'harassment': 3.5760862374445423e-05, 'harassment_threatening': 1.0533901331655215e-05, 'hate': 1.7152413533949584e-07, 'hate_threatening': 8.38202424802148e-07, 'illicit': None, 'illicit_violent': None, 'self_harm': 6.552371587531525e-07, 'self_harm_instructions': 3.247230552005931e-06, 'self_harm_intent': 2.563845782788121e-06, 'sexual': 1.994265039684251e-05, 'sexual_minors': 1.261122832829642e-07, 'violence': 0.0006444415776059031, 'violence_graphic': 2.028420385613572e-05, 'self-harm': 6.552371587531525e-07, 'sexual/minors': 1.261122832829642e-07, 'hate/threatening': 8.38202424802148e-07, 'violence/graphic': 2.028420385613572e-05, 'self-harm/intent': 2.563845782788121e-06, 'self-harm/instructions': 3.247230552005931e-06, 'harassment/threatening': 1.0533901331655215e-05}
AI Response is safe.


### Section 2.3 - Creating a speech response for customer

In [66]:
client = OpenAI(api_key=api_key)

# Translate chatbot reply
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": f"Translate the following text from English to {country_code}. Only return the translated text."},
        {"role": "user", "content": chatbot_reply}
    ],
    max_completion_tokens=500 
)

# Extract translated text
translated_reply = response.choices[0].message.content
print(translated_reply)

DataCamp offre una varietà di corsi e percorsi per aiutarti ad apprendere l'IA, a seconda dei tuoi interessi e obiettivi. Ecco alcune opzioni che puoi considerare:

1. **Ingegnere AI Associato per Sviluppatori**: Questo percorso professionale si concentra sull'integrazione dell'IA nelle applicazioni software, coprendo Modelli Linguistici di Grandi Dimensioni (LLM), ingegneria dei prompt, chatbot e altro ancora. Avrai anche la possibilità di esplorare strumenti come OpenAI API, Hugging Face, LangChain e Pinecone. Ha una durata di 30 ore ed è progettato per fornirti le competenze necessarie per avere successo come Ingegnere AI. Puoi trovare questo percorso [qui](https://www.datacamp.com/tracks/associate-ai-engineer-for-developers).

2. **Fondamenti di OpenAI**: Se sei particolarmente interessato a utilizzare l'API di OpenAI, questo percorso di competenze ti insegna come creare applicazioni AI come chatbot e sistemi di raccomandazione, incorporando anche tecniche avanzate di prompting. Qu

#### Preparing the spoken response

You've made it to the final step of the case study! Your task now is to convert the tailored, translated response into speech. Since the customer submitted their query via audio, they'll receive an audio reply in return—how cool is that?

The translated reply from the previous exercise is available as translated_reply.

In [67]:
client = OpenAI(api_key=api_key)

# Create the text-to-speech request
response = client.audio.speech.create(
    model="gpt-4o-mini-tts",
    voice="onyx",
    input=translated_reply
)

# Stream the response to an MP3 file
response.stream_to_file("output.mp3")

/tmp/ipykernel_286523/3112500670.py:11: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("output.mp3")
